In [1]:
from huggingface_hub import snapshot_download
from glob import glob
import pandas as pd

In [2]:
snapshot_download(repo_id="mesolitica/Malaysian-Reasoning", allow_patterns="data/*",
                 repo_type ='dataset', local_dir = './reasoning')

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

'/home/husein/ssd4/llama2/reasoning'

In [3]:

# !wget https://huggingface.co/datasets/mesolitica/Malaysian-SFT/resolve/main/extra/filter-qwq-500k-math.json

In [4]:
# !wget https://huggingface.co/datasets/mesolitica/Malay-Dialect-Reasoning/resolve/main/data/dialect_translation_train-00000-of-00001.parquet
# !wget https://huggingface.co/datasets/mesolitica/Malay-Dialect-Reasoning/resolve/main/data/reverse_dialect_translation_train-00000-of-00001.parquet

In [5]:
reasoning_en = 'You are going to enter reasoning mode, first you try to think step-by-step after that give the final answer.'
reasoning_ms = 'You are going to enter reasoning mode, first you try to think step-by-step in malay after that give the final answer.'

In [6]:
import json

with open('qwq-magpie.json') as fopen:
    d = json.load(fopen)
    
len(d)

7348

In [7]:
d[3]

{'problem': 'Design a wastewater treatment plant for a small city with a population of 50,000 people. The plant should be able to treat 10,000 m3 of wastewater per day, meeting the local discharge standards for BOD, COD, TSS, and ammonia nitrogen. Consider the plant layout, process selection, equipment sizing, and estimated capital and operating costs.',
 'keyword': 'Environmental Engineering',
 'outputs': "Designing a wastewater treatment plant for a small city with a population of 50,000 people and a daily wastewater flow of 10,000 cubic meters is a significant undertaking. The plant must treat the wastewater to meet local discharge standards for biochemical oxygen demand (BOD), chemical oxygen demand (COD), total suspended solids (TSS), and ammonia nitrogen. I'll need to consider the plant layout, process selection, equipment sizing, and estimated capital and operating costs.\n\nFirst, I need to understand the typical wastewater characteristics for a city of this size. Generally, do

In [8]:
instructions = []
for d_ in d:
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': d_['problem']},
        {'role': 'assistant', 'content': d_['outputs']}
    ])

In [9]:
with open('filter-qwq-500k-math.json') as fopen:
    d = json.load(fopen)
    
len(d)

34805

In [10]:
filter_d = [d_ for d_ in d if '**Final Answer**' in d_['response'][-1000:]]
len(filter_d)

31651

In [11]:
filter_d[-1]

{'prompt': 'As a technical writer for Finale music notation software, you\'re tasked with optimizing the layout of a complex musical score. The score consists of 12 staves, each with a unique set of notes, rests, and articulations. To minimize page turns and reduce clutter, you want to allocate the staves across multiple pages in a way that minimizes the total number of page breaks.\n\nAssuming each staff has a fixed height of 2.5 cm, and each page has a fixed height of 30 cm, you can fit a maximum of 12 staves per page. However, due to the complexity of the score, some staves have "anchor points" that must be aligned vertically across pages. These anchor points occur at specific rhythmic events, such as the start of a new measure or the onset of a note.\n\nGiven the following constraints:\n\n* Staff 1 has anchor points at measures 1, 5, and 9\n* Staff 2 has anchor points at measures 2, 6, and 10\n* Staff 3 has anchor points at measures 3, 7, and 11\n* Staff 4 has anchor points at meas

In [12]:
d = sorted(d, key = lambda x: len(x['response']), reverse = True)

In [13]:
for d_ in d[:15000]:
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': d_['prompt']},
        {'role': 'assistant', 'content': d_['response']}
    ])

In [14]:
len(instructions)

22348

In [15]:
glob('reasoning/data/*.parquet')

['reasoning/data/leetcode_hard-00000-of-00001.parquet',
 'reasoning/data/mallm-00000-of-00001.parquet',
 'reasoning/data/iium_confession_translation-00000-of-00001.parquet',
 'reasoning/data/dbp_dialect_translation-00000-of-00001.parquet',
 'reasoning/data/dbp_reverse_dialect_translation-00000-of-00001.parquet',
 'reasoning/data/tatabahasa-00000-of-00001.parquet',
 'reasoning/data/deepmath-00000-of-00001.parquet',
 'reasoning/data/maktabalbahri-00000-of-00001.parquet']

In [16]:
df = pd.read_parquet('reasoning/data/deepmath-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,None,None,None,8.0,Find the expectation \( \mathbb{E}[X_t] \) whe...,0,None,None,"Baiklah, saya perlu mencari jangkaan \( \mathb..."
1,None,None,None,8.0,Determine the limit of \( \frac{\pi(x)}{x} \) ...,0,None,None,"Baiklah, jadi saya mempunyai masalah ini: tent..."
2,None,None,None,8.0,Consider a sequence $(r_n)$ resulting from inf...,1,None,None,Jadi saya ada masalah di sini tentang flipping...
3,None,None,None,7.5,"Verify that for \(n = 2^kN\), where \(N\) is o...",0,None,None,"Okay, saya perlu mengesahkan identiti ini untu..."
4,None,None,None,8.0,Let $E \subset \mathbb{C}$ be a set such that ...,Yes,None,None,Jadi saya mempunyai masalah ini di sini. Ia be...


In [17]:
for i in range(len(df)):
    if df.iloc[i]['english'] is not None:
        system = reasoning_en
        content = df.iloc[i]['english']
    else:
        system = reasoning_ms
        content = df.iloc[i]['malay']
    
    instructions.append([
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': df.iloc[i]['content'].replace('\xa0', ' ')},
        {'role': 'assistant', 'content': content}
    ])
    
len(instructions)

25526

In [18]:
df = pd.read_parquet('dialect_translation_train-00000-of-00001.parquet')
df.head()

,content,solution,language,malay
0,Budak Usen tu maseh kolop lagi.,Budak Hussein itu masih belum bersunat.,johor,Bagus. Sekarang mari kita lakukan penerjemahan...
1,Sampai ati nasik belantok tu kau suruh aku makan.,Sampai hati nasi yang sudah tersimpan lama itu...,johor,Baguslah. Mari kita jalankan proses **analisis...
2,"Punyele bincau Mak Cik Jenab, tak sempat kawan...","Punyalah becok Mak Cik Zainab, tidak sempat sa...",johor,Baguslah. Kali ini kita akan melakukan penerje...
3,Salah siket gandang bunyi mulotnye.,"Salah sedikit sahaja, riuh rendah dan bising b...",johor,Okay. Biar saya jelaskan dengan **panjang leba...
4,"Layat siket dageng lembu tu, boleh kite baka.",Hiriskan sedikit daging lembu itu supaya boleh...,johor,Bagus. Mari kita lakukan analisis ini secara *...


In [19]:
import random

melayu = [
    'standard malay',
    'melayu baku',
    'standard melayu',
]

for i in range(len(df)):
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nterjemah ke {random.choice(melayu)}"},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
instructions[-1]

[{'role': 'system',
  'content': 'You are going to enter reasoning mode, first you try to think step-by-step in malay after that give the final answer.'},
 {'role': 'user',
  'content': 'Besok aku perari sawah die.\n\nterjemah ke melayu baku'},
 {'role': 'assistant',
  'content': 'Okay. Biar saya jelaskan **dengan lebih panjang dan terperinci**, langkah demi langkah, seperti yang diminta:\n\n## **Teks Dialek Asal:**\n\n**"Besok aku perari sawah die."**\n\n## **Pendekatan Analitikal Untuk Penterjemahan Ayat Dialek**\n\n### **Objektif**\n\nPenerbitan analitikal bertujuan untuk **memahami dan mentafsir** maksud serta **menyampaikan kembali** ayat dialek Melaka ke dalam **Bahasa Melayu Standard (BMS)** dengan tepat dan mengekalkan makna asalnya. Ini bukan hanya penterjemahan literal tetapi juga penerjemahan **konteks** dan **nada**.\n\n### **Langkah 1: Identifikasi Dialek dan Fungsi Ayat**\n\nAyat ini ditulis dalam **dialek Melaka**, sebuah varian Bahasa Melayu yang digunakan di negeri Mel

In [20]:
df = pd.read_parquet('reverse_dialect_translation_train-00000-of-00001.parquet')
df.head()

,content,solution,language,malay
0,"Dia demam biasa sahaja, belum demam kuat.","Demam biyase aje, belom melintap.",johor,"Baik, saya akan jelaskan dengan sangat terperi..."
1,"Apabila dapur ini sudah dibesarkan, nampaklah ...","Bile dah tambah dapo ni, nampaklah besibo ruma...",johor,"Baik, saya akan menerangkan dengan sangat terp..."
2,"Hendak menyambut maulud nabi ni, kita berpakat...","Nak nyambut mawlud nabi ni, kite pakat-pakat t...",johor,"Baik, saya akan menjelaskan secara terperinci ..."
3,"Habis buah limau itu, setelah pokok dipanjat b...","Gedi buah limau tu, panjat di budak.",johor,"Baiklah, saya akan menjelaskan secara terperin..."
4,Sia-sia sahaja kita ke sana sebab dia tidak ke...,"Handa saje kite ke sane, tak nampak muke pon die.",johor,"Baik, saya akan menjelaskan dengan sangat terp..."


In [21]:
for i in range(len(df)):
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nterjemah ke {df.iloc[i]['language']}"},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
instructions[-1]

[{'role': 'system',
  'content': 'You are going to enter reasoning mode, first you try to think step-by-step in malay after that give the final answer.'},
 {'role': 'user',
  'content': 'Lama tidak hujan, perigi kering kontang.\n\nterjemah ke melaka'},
 {'role': 'assistant',
  'content': 'Baiklah, saya akan terangkan langkah demi langkah yang sangat terperinci dan panjang bagaimana ayat Bahasa Melayu standard **"Lama tidak hujan, perigi kering kontang."** diterjemahkan ke dalam dialek Melaka. Saya akan menguraikan setiap perkataan, perubahan bunyi, struktur ayat, dan makna budaya supaya anda dapat memahami proses transformasi ayat ini dengan sangat mendalam.\n\n## Ayat Standard yang Diberikan:\n\n**Lama tidak hujan, perigi kering kontang.**\n\nMaksud ayat ini adalah: Sudah lama tidak turun hujan, jadi perigi air menjadi sangat kering.\n\n## Langkah 1: Memahami Konteks dan Maksud Ayat Standard\n\nAyat ini menggambarkan keadaan kekurangan air yang disebabkan oleh kurang hujan dalam tempo

In [22]:
df = pd.read_parquet('reasoning/data/leetcode_hard-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,1568,minimum-number-of-days-to-disconnect-island,Minimum Number of Days to Disconnect Island,Hard,You are given an `m x n` binary grid `grid` wh...,"```python\ndef pseudoPalindromicPaths(root, cn...",python,"Alright, so I've got this problem here about a...","Baiklah, jadi saya mempunyai masalah ini di si..."
1,1714,sum-of-special-evenly-spaced-elements-in-array,Sum Of Special Evenly-Spaced Elements In Array,Hard,You are given a **0-indexed** integer array `n...,```javascript\nfunction reorderSpaces(text) {\...,javascript,"Alright, I've got this problem here. So, I hav...","Baiklah, saya ada masalah ini di sini. Jadi, s..."
2,1326,minimum-number-of-taps-to-open-to-water-a-garden,Minimum Number of Taps to Open to Water a Garden,Hard,There is a one-dimensional garden on the x-axi...,```javascript\nfunction sumOfFlooredPairs(nums...,javascript,"Alright, I've got this problem about watering ...","Baiklah, saya mempunyai masalah ini tentang me..."
3,656,coin-path,Coin Path,Hard,You are given an integer array `coins` (**1-in...,```cpp\n#include <vector>\n#include <queue>\n#...,c++,"Alright, I've got this problem here. Let's see...","Baiklah, saya mempunyai masalah ini di sini. M..."
4,410,split-array-largest-sum,Split Array Largest Sum,Hard,Given an integer array `nums` and an integer `...,"```python\ndef splitArray(nums, m):\n left,...",python,"Alright, so I've got this problem here. I need...","Baiklah, jadi saya mempunyai masalah ini di si..."


In [23]:
for i in range(len(df)):
    instructions.append([
        {'role': 'system', 'content': reasoning_en},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nsolve the problem using {df.iloc[i]['language']}"},
        {'role': 'assistant', 'content': df.iloc[i]['english']}
    ])
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nsolve the problem using {df.iloc[i]['language']}"},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

37636

In [24]:
df = pd.read_parquet('reasoning/data/mallm-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,None,None,None,None,Manakah antara berikut merupakan catatan kontr...,A,None,None,"Sebagai penjawab, saya akan menjawab soalan in..."
1,None,None,None,None,Berikut adalah amalan perekodan perakaunan Zai...,D,None,None,Soalan ini berkaitan dengan amalan perekodan p...
2,None,None,None,None,Urus niaga manakah direkodkan di dalam Jurnal ...,A,None,None,Urus niaga yang dicatat dalam Jurnal Am adalah...
3,None,None,None,None,Berikut adalah maklumat yang diperoleh daripad...,B,None,None,Maklumat yang diberikan berkaitan dengan Dana ...
4,None,None,None,None,Manakah yang berikut merupakan akaun nyata?\n\...,B,None,None,Soalan ini bertanyakan tentang jenis-jenis aka...


In [25]:
for i in range(len(df)):
    q = df.iloc[i]['content']
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': q},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

44246

In [26]:
df = pd.read_parquet('reasoning/data/tatabahasa-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,None,None,None,None,Isi tempat kosong dengan jawapan yang paling s...,B,None,None,"Pertama, mari kita lihat ayat ini dengan lebih..."
1,None,None,None,None,Isi tempat kosong dengan jawapan yang paling s...,C,None,None,Amran ditawarkan sejumlah wang yang besar untu...
2,None,None,None,None,Isi tempat kosong dengan jawapan yang paling s...,D,None,None,Soalan ini meminta kita mengisi tempat kosong ...
3,None,None,None,None,Isi tempat kosong dengan pekataan berimbuhan y...,D,None,None,Soalan ini meminta saya untuk memilih pekataan...
4,None,None,None,None,Isi tempat kosong dalam ayat-ayat di bawah den...,A,None,None,Penolong Kanan Hal Ehwal Murid terpaksa melaku...


In [27]:
for i in range(len(df)):
    
    instructions.append([
        {'role': 'system', 'content': reasoning_ms},
        {'role': 'user', 'content': df.iloc[i]['content']},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

45605

In [28]:
df = pd.read_parquet('reasoning/data/maktabalbahri-00000-of-00001.parquet')
df.head()

,content,malay
0,Adakah benar syair “Tola’al Baru ‘Alaina” disy...,"Alhamdulillah, puji dan syukur kepada Ilahi at..."
1,"Asslamualaikum Ustaz, Recently, my employer in...","If you know for certain that it is money, for ..."
2,Assalamualaikum Dato’ Seri. Apa yang perlu say...,"Waalaikumusslam wbt.\n\nAlhamdulillah, segala..."
3,Apakah hukum menggunakan duit biasasiswa pelaj...,"Alhamdulillah, puji dan syukur kepada Ilahi at..."
4,What is the ruling of using one’s scholarship ...,"Alhamdulillah, praise and gratitude to Allah S..."


In [29]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [30]:
for i in range(len(df)):
    if lang_model.predict(df.iloc[i]['content'].replace('\n', ' '))[0][0] == '__label__english':
        system = reasoning_en
    else:
        system = reasoning_ms
    
    instructions.append([
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': df.iloc[i]['content'].replace('\xa0', ' ')},
        {'role': 'assistant', 'content': df.iloc[i]['malay'].replace('\xa0', ' ')}
    ])
    
len(instructions)

53084

In [31]:
df = pd.read_parquet('reasoning/data/iium_confession_translation-00000-of-00001.parquet')
df.head()

,id,slug,title,difficulty,content,solution,language,english,malay
0,None,None,None,None,Majikan seolah menjaga kebajikan kami dengan s...,The employer seems to take care of our well-be...,None,None,"-- Okay, so this Malay sentence is quite strai..."
1,None,None,None,None,Waktu nak berbuka belanja kfc makan dan sebaga...,"When its time to break our fast, the employers...",None,None,"Bagus, mari kita terokai ini secara berperingk..."
2,None,None,None,None,Kami kemudian bekerja separuh mati setiap hari...,We then worked so hard every day in order to a...,None,None,Baguslah. Sekarang mari kita pecahkan ayat ini...
3,None,None,None,None,Bukan untuk membangga diri tetapi saya juga ti...,"I'm not trying to brag, but I also made a lot ...",None,None,"Okay, so I've got this Malay sentence here tha..."
4,None,None,None,None,Itu pada awalnya.,That was on the beginning.,None,None,"Okay, biar saya analisis langkah demi langkah ..."


In [32]:
english = [
    'standard english',
    'inggeris baku',
    'standard inggeris',
]

for i in range(len(df)):
    if lang_model.predict(df.iloc[i]['content'].replace('\n', ' '))[0][0] == '__label__english':
        system = reasoning_en
    else:
        system = reasoning_ms
    
    instructions.append([
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': df.iloc[i]['content'] + f"\n\nterjemah ke {random.choice(english)}"},
        {'role': 'assistant', 'content': df.iloc[i]['malay']}
    ])
    
len(instructions)

56065

In [33]:
instructions[-2]

[{'role': 'system',
  'content': 'You are going to enter reasoning mode, first you try to think step-by-step in malay after that give the final answer.'},
 {'role': 'user',
  'content': 'Jadi malam tu aku tidur dengan pakcik aku.\n\nterjemah ke standard inggeris'},
 {'role': 'assistant',
  'content': 'Okay, mari kita dekati ini langkah demi langkah. Ayat Melayu tempatan adalah:\n\n**"Jadi malam tu aku tidur dengan pakcik aku."**\n\nMatlamat kami adalah untuk menukarnya menjadi Bahasa Inggeris standard yang natural, memastikan ia menghormati sensitiviti budaya dan norma-norma sosial yang berkaitan dengan hubungan kekeluargaan.\n\n## Langkah Demi Langkah: Terjemah Ayat Melayu Tempatan ke Bahasa Inggeris Standard\n\n### Langkah 1: Fahami Konteks dan Nada Ayat\n\nPertama, kita perlu memahami konteks di mana ayat ini digunakan. Ayat ini nampaknya merujuk kepada situasi di mana penutur tidur di rumah pakcik mereka, mungkin kerana mereka tinggal bersamanya atau kerana ada acara keluarga. Nada

In [34]:
with open('combined-malaysian-reasoning.jsonl', 'w') as fopen:
    for i in instructions:
        fopen.write(f'{json.dumps(i)}\n')

In [35]:
!ls -lh combined-malaysian-reasoning.jsonl

-rw-rw-r-- 1 husein husein 757M Mei  25 15:00 combined-malaysian-reasoning.jsonl


In [37]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="combined-malaysian-reasoning.jsonl",
    path_in_repo="combine/combined-malaysian-reasoning.jsonl",
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

combined-malaysian-reasoning.jsonl:   0%|          | 0.00/793M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/53936ee51e7697eb8b8126ee2074c6030018c46d', commit_message='Upload combine/combined-malaysian-reasoning.jsonl with huggingface_hub', commit_description='', oid='53936ee51e7697eb8b8126ee2074c6030018c46d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-SFT', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-SFT'), pr_revision=None, pr_num=None)